In [37]:
import pandas as pd
import talib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import cufflinks as cf
from plotly.offline import iplot
from scipy.stats import linregress
import plotly.graph_objects as go
import scipy
import math
import pandas_ta as ta
cf.go_offline()  # will make cufflinks offline
cf.set_config_file(offline=False, world_readable=True)

ticker = "AAPL"
frequency = "daily"
# perhaps set to external harddrive to accomodate large amount of data
data_path = Path(os.getcwd())
print(data_path)
folder = "company stock price {}".format(frequency)
file = ticker+".csv"
full_path = data_path/"stock price data"/folder/file

ticker_data = pd.read_csv(full_path).drop(
    ["label", "changeOverTime", "unadjustedVolume", "Symbol"], axis=1).sort_values(by="date").reset_index(drop=True)
ticker_data

c:\Users\fuway\Desktop\stocks


,date,open,high,low,close,adjClose,volume,change,changePercent,vwap
0,2010-01-04,7.6225,7.6607,7.5850,7.6432,6.496293,493719800,0.0207,0.27156,7.63
1,2010-01-05,7.6643,7.6996,7.6161,7.6564,6.507526,601901216,-0.0079,-0.10308,7.67
2,2010-01-06,7.6564,7.6868,7.5268,7.5346,6.404015,552158376,-0.1218,-1.59000,7.61
3,2010-01-07,7.5625,7.5714,7.4661,7.5207,6.392176,477129296,-0.0418,-0.55273,7.51
4,2010-01-08,7.5107,7.5714,7.4664,7.5707,6.434673,447872124,0.0600,0.79886,7.53
...,...,...,...,...,...,...,...,...,...,...
3397,2023-07-05,191.5650,192.9800,190.6200,191.3300,191.330002,46896861,-0.2350,-0.12267,191.58
3398,2023-07-06,189.8400,192.0200,189.2000,191.8100,191.809998,45155523,1.9700,1.04000,190.82
3399,2023-07-07,191.4100,192.6700,190.2400,190.6800,190.679993,46757498,-0.7300,-0.38138,191.42
3400,2023-07-10,189.2600,189.9900,187.0350,188.6100,188.610001,59912163,-0.6500,-0.34344,188.36


In [38]:
#basic indicators
def spread_features(df):
    # Calculate the daily spread
    df['spread'] = df['high'] - df['low']

    # Calculate the 5-day average spread
    df['5_day_avg_spread'] = df['spread'].rolling(window=5).mean()

    return df


def volume_features(df):
    # Calculate the 5-day average volume
    df['5_day_avg_volume'] = df['volume'].rolling(window=5).mean()
    return df


ticker_data = volume_features(spread_features(ticker_data))

ticker_data["EMA_5"] = talib.EMA(
    ticker_data.sort_values(by="date")["close"], timeperiod=5)
ticker_data["EMA_10"] = talib.EMA(
    ticker_data.sort_values(by="date")["close"], timeperiod=10)
ticker_data["EMA_20"] = talib.EMA(
    ticker_data.sort_values(by="date")["close"], timeperiod=20)
ticker_data["EMA_50"] = talib.EMA(
    ticker_data.sort_values(by="date")["close"], timeperiod=50)
ticker_data["EMA_200"] = talib.EMA(
    ticker_data.sort_values(by="date")["close"], timeperiod=200)


def identify_trend(row, df, ema_column, days):
    idx = row.name
    if idx < days - 1:  # If there aren't enough previous days to check
        return 'None'

    is_bearish = all(df.loc[idx - i, 'close'] <
                     df.loc[idx - i, ema_column] for i in range(days))
    is_bullish = all(df.loc[idx - i, 'close'] >
                     df.loc[idx - i, ema_column] for i in range(days))

    if is_bearish:
        return 'Bearish'
    elif is_bullish:
        return 'Bullish'
    else:
        return 'None'


# Identify trends
ticker_data['short_trend'] = ticker_data.apply(
    identify_trend, df=ticker_data, ema_column='EMA_5', days=5, axis=1)
ticker_data['mid_trend'] = ticker_data.apply(
    identify_trend, df=ticker_data, ema_column='EMA_20', days=10, axis=1)
ticker_data['long_trend'] = ticker_data.apply(
    identify_trend, df=ticker_data, ema_column='EMA_200', days=30, axis=1)

ticker_data

,date,open,high,low,close,adjClose,volume,change,changePercent,vwap,...,5_day_avg_spread,5_day_avg_volume,EMA_5,EMA_10,EMA_20,EMA_50,EMA_200,short_trend,mid_trend,long_trend
0,2010-01-04,7.6225,7.6607,7.5850,7.6432,6.496293,493719800,0.0207,0.27156,7.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,2010-01-05,7.6643,7.6996,7.6161,7.6564,6.507526,601901216,-0.0079,-0.10308,7.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,2010-01-06,7.6564,7.6868,7.5268,7.5346,6.404015,552158376,-0.1218,-1.59000,7.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,2010-01-07,7.5625,7.5714,7.4661,7.5207,6.392176,477129296,-0.0418,-0.55273,7.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
4,2010-01-08,7.5107,7.5714,7.4664,7.5707,6.434673,447872124,0.0600,0.79886,7.53,...,0.1059,514556162.4,7.585120,NaN,NaN,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,2023-07-05,191.5650,192.9800,190.6200,191.3300,191.330002,46896861,-0.2350,-0.12267,191.58,...,2.2260,52197524.0,191.011621,189.205505,186.045838,178.549025,161.971032,Bullish,Bullish,Bullish
3398,2023-07-06,189.8400,192.0200,189.2000,191.8100,191.809998,45155523,1.9700,1.04000,190.82,...,2.3300,50985268.4,191.277747,189.679049,186.594806,179.069063,162.267937,Bullish,Bullish,Bullish
3399,2023-07-07,191.4100,192.6700,190.2400,190.6800,190.679993,46757498,-0.7300,-0.38138,191.42,...,2.5900,51067306.4,191.078498,189.861040,186.983872,179.524394,162.550644,None,Bullish,Bullish
3400,2023-07-10,189.2600,189.9900,187.0350,188.6100,188.610001,59912163,-0.6500,-0.34344,188.36,...,2.5370,46036048.6,190.255665,189.633578,187.138741,179.880692,162.809941,None,Bullish,Bullish


In [39]:
#candle indicators
# 1 candle pattern
def doji(row, f_price=0.001, l_leg=[0.1, 0.4], dg=[4, 0.1, 0.6]):
    open_price = row['open']
    close_price = row['close']
    high_price = row['high']
    low_price = row['low']

    upper_shadow = high_price - max(open_price, close_price)
    lower_shadow = min(open_price, close_price) - low_price
    real_body = abs(open_price - close_price)

    # 4-price Doji
    if (high_price-low_price)/open_price <= f_price:
        return "4-price Doji"
    # Long-Legged Doji
    elif real_body <= l_leg[0] * (high_price - low_price) and upper_shadow > l_leg[1] * (high_price - low_price) and lower_shadow > l_leg[1] * (high_price - low_price):
        if row["spread"] > row["5_day_avg_spread"]:
            return 'Long-Legged Doji'
        else:
            return "Standard Doji"

    # Dragonfly Doji
    elif dg[0] * real_body <= lower_shadow and upper_shadow <= dg[1] * (high_price - low_price) and lower_shadow > dg[2] * (high_price - low_price):
        return 'Dragonfly Doji'

    # Gravestone Doji
    elif dg[0] * real_body <= upper_shadow and lower_shadow <= dg[1] * (high_price - low_price) and upper_shadow > dg[2] * (high_price - low_price):
        return 'Gravestone Doji'
    else:
        return "None"


def hammer(row):
    doji = row["doji"]
    short_trend = row["short_trend"]
    if short_trend == "Bearish" and doji == "Dragonfly Doji":
        return "hammer"
    elif short_trend == "Bullish" and doji == "Gravestone Doji":
        return "inverted hammer"
    return "None"


def marobuza(row, body=0.9):
    spread = row.spread
    open_price = row.open
    close_price = row.close

    if open_price > close_price and (open_price - close_price) > body*spread:
        return "bullish marobuza"
    elif open_price < close_price and (close_price - open_price) > body*spread:
        return "bearish marobuza"
    return "None"

# 2 candle pattern


def engulfing(df, extreme=True):

    engulfing = []

    for i in range(1, len(df)):
        # Previous day data
        prev_open = df.loc[i-1, 'open']
        prev_close = df.loc[i-1, 'close']
        prev_high = df.loc[i-1, "high"]
        prev_low = df.loc[i-1, "low"]

        # Current day data
        curr_open = df.loc[i, 'open']
        curr_close = df.loc[i, 'close']

        # Bullish engulfing condition
        if (prev_close < prev_open) and (curr_open < prev_close) and (curr_close > prev_open):
            if extreme:
                if curr_close > prev_high and curr_open < prev_low:
                    engulfing.append('bullish engulfing')
                else:
                    engulfing.append("None")
            else:
                engulfing.append("bullish engulfing")
        # Bearish engulfing condition
        elif (prev_close > prev_open) and (curr_open > prev_close) and (curr_close < prev_open):
            if extreme:
                if curr_close < prev_low and curr_open < prev_high:
                    engulfing.append('bearish engulfing')
                else:
                    engulfing.append("None")
            else:
                engulfing.append('bearish engulfing')
        else:
            engulfing.append("None")

    # Account for the first row, which can't be evaluated for engulfing pattern
    engulfing = ["None"] + engulfing

    df['engulfing'] = engulfing

    return df


def harami(df, extreme=True):

    # extreme version

    harami = []

    for i in range(1, len(df)):
        # Previous day data
        prev_open = df.loc[i-1, 'open']
        prev_close = df.loc[i-1, 'close']

        # Current day data
        curr_open = df.loc[i, 'open']
        curr_close = df.loc[i, 'close']
        curr_high = df.loc[i, "high"]
        curr_low = df.loc[i, "low"]

        # Bullish engulfing condition
        if (prev_close < prev_open) and (curr_close > curr_open) and (curr_open > prev_close) and (curr_close < prev_open):
            if extreme:
                if curr_high < prev_open and curr_low > prev_close:
                    harami.append('bullish harami')
                else:
                    harami.append("None")
            else:
                harami.append('bullish harami')
        # Bearish engulfing condition
        elif (prev_close > prev_open) and (curr_close < curr_open) and (curr_open < prev_close) and (curr_close > prev_open):
            if extreme:
                if curr_high < prev_close and curr_low > prev_open:
                    harami.append('bearish harami')
                else:
                    harami.append("None")
            else:
                harami.append('bearish harami')
        else:
            harami.append("None")

    # Account for the first row, which can't be evaluated for engulfing pattern
    harami = ["None"] + harami

    df['harami'] = harami

    return df


def piercing_darkcloud(df):

    piercing_darkcloud = []

    for i in range(1, len(df)):
        # Previous day data
        prev_open = df.loc[i-1, 'open']
        prev_close = df.loc[i-1, 'close']
        prev_midpoint = (prev_open + prev_close) / 2

        # Current day data
        curr_open = df.loc[i, 'open']
        curr_close = df.loc[i, 'close']

        # Piercing Line condition
        if (prev_close < prev_open) and (curr_open < prev_close) and (curr_close > prev_midpoint) and (curr_close < prev_open):
            piercing_darkcloud.append('bullish piercing')

        # Dark Cloud Cover condition
        elif (prev_close > prev_open) and (curr_open > prev_close) and (curr_close < prev_midpoint) and (curr_close > prev_open):
            piercing_darkcloud.append('bearish dark cloud')

        else:
            piercing_darkcloud.append("None")

    # Account for the first row, which can't be evaluated for patterns
    piercing_darkcloud = ["None"] + piercing_darkcloud

    df['piercing_darkcloud'] = piercing_darkcloud

    return df


def kicker(df):
    kicker = []

    for i in range(1, len(df)):
        # Previous day data
        prev_open = df.loc[i-1, 'open']
        prev_close = df.loc[i-1, 'close']

        # Current day data
        curr_open = df.loc[i, 'open']
        curr_close = df.loc[i, 'close']
        curr_vol = df.loc[i, "volume"]
        avg_vol = df.loc[i, "5_day_avg_volume"]

        # Bullish Kicker condition, including confirmation by volume
        if (prev_close < prev_open) and (curr_open > prev_open) and (curr_close > curr_open) and (curr_vol > avg_vol):
            kicker.append('bullish kicker')

        # Bearish Kicker condition, including confirmation by volume
        elif (prev_close > prev_open) and (curr_open < prev_open) and (curr_close < curr_open) and (curr_vol > avg_vol):
            kicker.append('bearish kicker')

        else:
            kicker.append("None")

    # Account for the first row, which can't be evaluated for patterns
    kicker = ["None"] + kicker

    df['kicker'] = kicker

    return df

# 3 candle pattern


def star(df, s_l=[0.3, 0.6]):
    star = ["None" for i in range(len(df))]

    for i in range(2, len(df)):
        # Previous day data
        first_open = df.loc[i-2, 'open']
        first_close = df.loc[i-2, 'close']
        first_body = abs(first_close-first_open)
        first_spread = df.loc[i-2, "high"] - df.loc[i-2, "low"]

        second_open = df.loc[i-1, 'open']
        second_close = df.loc[i-1, 'close']
        second_spread = df.loc[i-1, "high"] - df.loc[i-1, "low"]
        second_body = abs(second_open - second_close)

        # Current day data
        third_open = df.loc[i, 'open']
        third_close = df.loc[i, 'close']
        volume = df.loc[i, "volume"]
        avg_volume = df.loc[i, "5_day_avg_volume"]

        # Evening Star condition
        if (first_close > first_open) and first_body > s_l[1]*first_spread:
            if (second_open > first_close) and (second_close > second_open) and (second_body < s_l[0]*second_spread):
                if (third_close < third_open) and (third_close < (first_open + (first_close - first_open)/2)):
                    if volume > avg_volume:
                        star[i] = 'bearish star'

        # Morning Star condition
        elif (first_close < first_open) and first_body > s_l[1]*first_spread:
            if (second_open < first_close) and (second_close < second_open) and (second_body < s_l[0]*second_spread):
                if (third_close > third_open) and (third_close > (first_open + (first_close - first_open)/2)):
                    if volume > avg_volume:
                        star[i] = 'bullish star'

    df['star'] = star

    return df


def three_fr(df, l=0.6):
    three = ["None" for i in range(len(df))]

    for i in range(len(df)):
        for in_between_count in [3, 4, 5]:
            if i - (in_between_count + 1) < 0:
                continue

            # First candlestick data
            first_open = df.loc[i - (in_between_count + 1), 'open']
            first_close = df.loc[i - (in_between_count + 1), 'close']
            first_spread = df.loc[i - (in_between_count + 1), 'high'] - \
                df.loc[i - (in_between_count + 1), 'low']
            first_body = abs(first_open - first_close)
            # Last candlestick data
            last_open = df.loc[i, 'open']
            last_close = df.loc[i, 'close']
            last_spread = df.loc[i, 'high'] - df.loc[i, 'low']
            last_body = abs(last_open - last_close)

            # Check for Falling Three Methods
            if (first_close < first_open) and (first_body > l*first_spread) and (last_close < last_open) and (last_body > last_spread * l) and (last_close < first_close):
                for j in range(i - in_between_count, i):
                    # In-between candles are bullish and within the range of the first candle
                    if df.loc[j, 'close'] > df.loc[j, 'open'] and df.loc[j, 'close'] > df.loc[i - (in_between_count + 1), 'close'] and df.loc[j, 'close'] < df.loc[i - (in_between_count + 1), 'open']:
                        continue
                    else:
                        break
                else:
                    three[i] = 'bearish three'
                    break

            # Check for Rising Three Methods
            if (first_close > first_open) and (first_body > l*first_spread) and (last_close > last_open) and (last_body > last_spread * l) and (last_close > first_close):
                for j in range(i - in_between_count, i):
                    # In-between candles are bearish and within the range of the first candle
                    if df.loc[j, 'close'] < df.loc[j, 'open'] and df.loc[j, 'close'] < df.loc[i - (in_between_count + 1), 'close'] and df.loc[j, 'close'] > df.loc[i - (in_between_count + 1), 'open']:
                        continue
                    else:
                        break
                else:
                    three[i] = 'bullish three'
                    break

    df['three_fr'] = three

    return df


def three_sc(df, l=0.4):
    sc = ["None" for i in range(len(df))]

    for i in range(2, len(df)):
        # Previous candlesticks data
        first_open = df.loc[i-2, 'open']
        first_close = df.loc[i-2, 'close']
        first_body = abs(first_open - first_close)

        second_open = df.loc[i-1, 'open']
        second_close = df.loc[i-1, 'close']
        second_body = abs(second_open - second_close)

        third_open = df.loc[i, 'open']
        third_close = df.loc[i, 'close']
        third_body = abs(third_open - third_close)
        # Three White Soldiers condition
        if (first_close > first_open) and (second_close > second_open) and (third_close > third_open) and \
           (second_open > first_open) and (second_open < first_close) and (second_close > first_close) and \
           (third_open > second_open) and (third_open < second_close) and (third_close > second_close) and first_body > l and second_body > l and third_body > l:
            sc[i] = 'bullish soldiers'

        # Three Black Crows condition
        elif (first_close < first_open) and (second_close < second_open) and (third_close < third_open) and \
             (second_open > first_close) and (second_open < first_open) and (second_close < first_close) and \
             (third_open > second_close) and (third_open < second_open) and (third_close < second_close) and first_body > l and second_body > l and third_body > l:
            sc[i] = 'bearish crows'

    df['three_sc'] = sc

    return df


def tasuki(df):
    tasuki = ["None" for i in range(len(df))]

    for i in range(2, len(df)):
        # Previous candlesticks data
        first_open = df.loc[i-2, 'open']
        first_close = df.loc[i-2, 'close']

        second_open = df.loc[i-1, 'open']
        second_close = df.loc[i-1, 'close']

        third_open = df.loc[i, 'open']
        third_close = df.loc[i, 'close']

        # Bullish Tasuki Gap condition
        if (first_close > first_open) and (second_close > second_open) and (third_close < third_open) and \
           (second_open > first_close) and (third_open > second_open) and (third_open < second_close) and \
           (third_close > first_close) and (third_close < second_open):
            tasuki[i] = 'bullish tasuki'

        # Bearish Tasuki Gap condition
        elif (first_close < first_open) and (second_close < second_open) and (third_close > third_open) and \
             (second_open < first_close) and (third_open < second_open) and (third_open > second_close) and \
             (third_close < first_close) and (third_close > second_open):
            tasuki[i] = 'bearish tasuki'

    df['tasuki'] = tasuki

    return df

# more than 3 bars


def island(df, i_l=range(2, 7)):
    island = ["None" for i in range(len(df))]

    for island_len in i_l:  # Loop through possible island lengths of 3 to 5 bars
        for i in range(island_len, len(df) - 1):

            # Price and volume data for the island
            # island_closes = df.loc[i-island_len:i-1, 'close'].values
            island_volumes = df.loc[i-island_len:i-1, 'volume'].values
            island_low = min(df.loc[i-island_len:i-1, 'low'].values)
            island_high = max(df.loc[i-island_len:i-1, 'high'].values)

            # Price data for the gap candles before and after the island
            if i-island_len-1 >= 0:  # Ensure we are not accessing negative indices
                prev_close = df.loc[i-island_len-1, 'close']
                prev_vol = df.loc[i-island_len-1, "5_day_avg_volume"]
                prev_trend = df.loc[i-island_len-1, "short_trend"]
                # gap high low
                gap_high = max(df.loc[i, 'high'],
                               df.loc[i-island_len-1, "high"])
                gap_low = min(df.loc[i, 'low'], df.loc[i-island_len-1, "low"])
                # Check for volume increase compared to preceding trend (approximated by 5 days average volume)
                volume_increase = all(vol > prev_vol for vol in island_volumes)
            else:
                prev_close = None

            # Island Reversal Top condition
            if prev_close and prev_trend == "Bullish" and (gap_high < island_low) and volume_increase:
                island[i] = 'bearish island'

            # Island Reversal Bottom condition
            elif prev_close and prev_trend == "Bearish" and (gap_low > island_high) and volume_increase:
                island[i] = 'bullish island'

    df['island'] = island

    return df


def hikkake(df, drift_candles=range(1, 4)):
    hikkake = ["None" for i in range(len(df))]
    for k in drift_candles:
        for i in range(3 + k, len(df)):
            # Extract the high, low, open, and close values for the initial days in the pattern
            high0, low0, open0, close0 = df.loc[i -
                                                (3+k), ['high', 'low', "open", "close"]]
            high1, low1, open1, close1 = df.loc[i -
                                                (2+k), ['high', 'low', "open", "close"]]
            high2, low2, close2, open2 = df.loc[i -
                                                (1+k), ['high', 'low', "close", "open"]]
            close_current = df.loc[i, 'close']

            # Bullish Hikkake
            inside_bar = (max(open0, close0) > high1) and (
                min(open0, close0) < low1)
            false_breakout_bullish = (close2 < low0)
            drifts_below = all((max(df.loc[i-j, "close"], df.loc[i-j, "open"]) > min(close2, open2)) and (
                max(df.loc[i-j, "close"], df.loc[i-j, "open"]) < max(close2, open2)) for j in range(1, k + 1))
            final_candle_above = close_current > high1

            # Bearish Hikkake
            false_breakout_bearish = (close2 > high0)
            drifts_above = all((min(df.loc[i-j, "close"], df.loc[i-j, "open"]) < max(close2, open2)) and (
                min(df.loc[i-j, "close"], df.loc[i-j, "open"]) > min(close2, open2)) for j in range(1, k + 1))
            final_candle_below = close_current < low1

            # # If all conditions are met, label the pattern as Hikkake
            if inside_bar:
                if false_breakout_bullish and drifts_below and final_candle_above:
                    hikkake[i] = 'bullish hikkake'
                elif false_breakout_bearish and drifts_above and final_candle_below:
                    hikkake[i] = 'bearish hikkake'

    df['hikkake'] = hikkake

    return df

ticker_data["doji"] = ticker_data.apply(doji, axis=1)
ticker_data["hammer"] = ticker_data.apply(hammer, axis=1)
ticker_data["marobuza"] = ticker_data.apply(marobuza, axis=1)
ticker_data = hikkake(island(tasuki(three_sc(
    three_fr(star(kicker(piercing_darkcloud(harami(engulfing(ticker_data))))))))))
ticker_data[~(ticker_data["hikkake"] == "None")].iplot(kind="candle")

In [40]:

#pattern indicators
# def find_levels( 
#         price: np.array, atr: float, # Log closing price, and log atr 
#         first_w: float = 0.5, 
#         atr_mult: float = 3.0, 
#         prom_thresh: float = 0.1
# ):

#     # Setup weights
#     last_w = 1.0
#     w_step = (last_w - first_w) / len(price)
#     weights = first_w + np.arange(len(price)) * w_step
#     weights[weights < 0] = 0.0

#     # Get kernel of price. 
#     kernal = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

#     # Construct market profile
#     min_v = np.min(price)
#     max_v = np.max(price)
#     step = (max_v - min_v) / 200
#     price_range = np.arange(min_v, max_v, step)
#     pdf = kernal(price_range) # Market profile

#     # Find significant peaks in the market profile
#     pdf_max = np.max(pdf)
#     prom_min = pdf_max * prom_thresh

#     peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)
#     levels = [] 
#     for peak in peaks:
#         levels.append(np.exp(abs(price_range[peak])))

#     return levels, peaks, props, price_range, pdf, weights




# def support_resistance_levels(
#         data: pd.DataFrame, lookback: int, 
#         first_w: float = 0.5, atr_mult:float=3, prom_thresh:float =0.1
# ):

#     # Get log average true range, 
#     atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

#     all_levels = [None] * len(data)
#     for i in range(lookback, len(data)):
#         i_start  = i - lookback
#         vals = np.log(data.iloc[i_start+1: i+1]['close'].to_numpy())
#         levels, peaks, props, price_range, pdf, weights= find_levels(vals, atr.iloc[i], first_w, atr_mult, prom_thresh)
#         all_levels[i] = levels
        
#     return all_levels

# def sr_penetration_signal(data: pd.DataFrame, levels: list):
#     signal = np.zeros(len(data))
#     curr_sig = 0.0
#     close_arr = data['close'].to_numpy()
#     for i in range(1, len(data)):
#         if levels[i] is None:
#             continue

#         last_c = close_arr[i - 1]
#         curr_c = close_arr[i]

        
#         for level in levels[i]:
#             if curr_c > level and last_c <= level: # Close cross above line
#                 curr_sig = 1.0
#             elif curr_c < level and last_c >= level: # Close cross below line
#                 curr_sig = -1.0

#         signal[i] = curr_sig
#     return signal
# sr = support_resistance_levels(data = ticker_data, lookback=60)

In [41]:
#pattern indicators
def pivotid(df1, l, n1, n2):  # n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow = 1
    pividhigh = 1
    for i in range(l-n1, l+n2+1):
        if (df1.low[l] > df1.low[i]):
            pividlow = 0
        if (df1.high[l] < df1.high[i]):
            pividhigh = 0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def check_trend_line(support: bool, pivot: int, slope: float, y: np.array):
    # compute sum of differences between line and prices, 
    # return negative val if invalid 
    
    # Find the intercept of the line going through pivot point with given slope
    intercept = -slope * pivot + y.loc[pivot]
    line_vals = slope * y.index + intercept
     
    diffs = line_vals - y
    
    # Check to see if the line is valid, return -1 if it is not valid.
    if support and diffs.max() > 1e-5:
        return -1.0
    elif not support and diffs.min() < -1e-5:
        return -1.0

    # Squared sum of diffs between data and line 
    err = (diffs ** 2.0).sum()
    return err;


def optimize_slope(support: bool, pivot:int , init_slope: float, y: np.array):
    
    # Amount to change slope by. Multiplyed by opt_step
    slope_unit = (y.max() - y.min()) / len(y) 
    
    # Optmization variables
    opt_step = 1.0
    min_step = 0.0001
    curr_step = opt_step # current step
    
    # Initiate at the slope of the line of best fit
    best_slope = init_slope
    best_err = check_trend_line(support, pivot, init_slope, y)
    assert(best_err >= 0.0) # Shouldn't ever fail with initial slope

    get_derivative = True
    derivative = None
    while curr_step > min_step:

        if get_derivative:
            # Numerical differentiation, increase slope by very small amount
            # to see if error increases/decreases. 
            # Gives us the direction to change slope.
            slope_change = best_slope + slope_unit * min_step
            test_err = check_trend_line(support, pivot, slope_change, y)
            derivative = test_err - best_err;
            
            # If increasing by a small amount fails, 
            # try decreasing by a small amount
            if test_err < 0.0:
                slope_change = best_slope - slope_unit * min_step
                test_err = check_trend_line(support, pivot, slope_change, y)
                derivative = best_err - test_err

            if test_err < 0.0: # Derivative failed, give up
                raise Exception("Derivative failed. Check your data. ")

            get_derivative = False

        if derivative > 0.0: # Increasing slope increased error
            test_slope = best_slope - slope_unit * curr_step
        else: # Increasing slope decreased error
            test_slope = best_slope + slope_unit * curr_step
        

        test_err = check_trend_line(support, pivot, test_slope, y)
        if test_err < 0 or test_err >= best_err: 
            # slope failed/didn't reduce error
            curr_step *= 0.5 # Reduce step size
        else: # test slope reduced error
            best_err = test_err 
            best_slope = test_slope
            get_derivative = True # Recompute derivative
    
    # Optimize done, return best slope and intercept
    return (best_slope, -best_slope * pivot + y[pivot])

def fit_trendlines_high_low(high: np.array, low: np.array, close: np.array):
    x = close.index
    coefs = np.polyfit(x, close, 1)
    # coefs[0] = slope,  coefs[1] = intercept
    line_points = coefs[0] * x + coefs[1]
    upper_pivot = (high - line_points).argmax() + high.index[0]
    lower_pivot = (low - line_points).argmin() + low.index[0]
    
    support_coefs = optimize_slope(True, lower_pivot, coefs[0], low)
    resist_coefs = optimize_slope(False, upper_pivot, coefs[0], high)

    return (support_coefs, resist_coefs)

def fit_all_high_low(df, lookback=[25,100]):
    # Take natural log of data to resolve price scaling issues
    data = np.log(df[["high", "low", "close", "open"]])

    support_slope = [np.nan] * len(data)
    # support_interc = [np.nan] * len(data)
    resist_slope = [np.nan] * len(data)
    # resist_interc = [np.nan] * len(data)
    for j in range(len(lookback)):
        for i in range(lookback[j] - 1, len(data)):
            candles = data.iloc[i - lookback[j] + 1: i + 1]
            support_coefs, resist_coefs =  fit_trendlines_high_low(candles['high'], 
                                                                    candles['low'], 
                                                                    candles['close'])
            support_slope[i] = support_coefs[0]
            resist_slope[i] = resist_coefs[0]
            # support_interc[i] = support_coefs[1]
            # resist_interc[i] = resist_coefs[1]
        if j == 0:
            df['short_support_slope'] = support_slope
            # df["short_support_interc"] = support_interc
            df['short_resist_slope'] = resist_slope
            # data['short_resist_interc'] = resist_interc
        else:
            df['long_support_slope'] = support_slope
            # df["long_support_interc"] = support_interc
            df['long_resist_slope'] = resist_slope
            # df['long_resist_interc'] = resist_interc
    return df

def Tri_Rec_Broad(df, lookback=25, tri=[0.0001, 0.0008], rec=[0.0001, 0.00001], broad=0.0008):
    df["Triangle"] = "None"
    df["Rectangle"] = "None"
    df["Broaden"] = "None"
    for candleid in range(lookback, len(df)):
        slmin = df.loc[candleid, "short_support_slope"]
        slmax = df.loc[candleid, "short_resist_slope"]
        if slmin >= tri[1] and abs(slmax) <= tri[0]:
            df.loc[candleid, "Triangle"] = "Ascending T"
        elif abs(slmin) <= tri[0] and slmax <= -tri[1]:
            df.loc[candleid, "Triangle"] = "Descending T"
        elif slmin >= tri[1] and slmax <= -tri[1]:
            df.loc[candleid, "Triangle"] = "Symmetric T"
        if abs(slmin) <= rec[0] and abs(slmax) <= rec[0] and abs(slmin-slmax) <= rec[1]:
            df.loc[candleid, "Rectangle"] = "Flat R"
        elif slmin < -rec[0] and slmax < -rec[0] and abs(slmin-slmax) <= rec[1]:
            df.loc[candleid, "Rectangle"] = "Down R"
        elif slmin > rec[0] and slmax > rec[0] and abs(slmin-slmax) <= rec[1]:
            df.loc[candleid, "Rectangle"] = "Up R"
        if slmin < -broad and slmax > broad:
            df.loc[candleid, "Broaden"] = "Broaden"
    return df

def trendline_breakout(df, lookback=[25,100], threshold = 0.005):
    data = np.log(df[["high", "low", "close", "open"]])

    for j in range(len(lookback)):
        s_tl = np.zeros(len(data))
        s_tl[:] = np.nan

        r_tl = np.zeros(len(data))
        r_tl[:] = np.nan

        sig = np.zeros(len(data))
        for i in range(lookback[j], len(data)):
            candles = data.iloc[i - lookback[j]: i]
            support_coefs, resist_coefs =  fit_trendlines_high_low(candles['high'], 
                                                                    candles['low'], 
                                                                    candles['close'])

            # Find current value of line, projected forward to current bar
            s_tl[i] = support_coefs[1] + (i) * support_coefs[0] 
            r_tl[i] = resist_coefs[1] + (i) * resist_coefs[0] 

            if (data.loc[i,"close"] - r_tl[i])/data.loc[i,"close"] > threshold:
                sig[i] = 1.0
            elif (s_tl[i]-data.loc[i,"close"])/data.loc[i,"close"] > threshold:
                sig[i] = -1.0
        if j==0:
            df['short_support'] = np.exp(s_tl)
            df['short_resist'] = np.exp(r_tl)
            df['short_trend_break_signal'] = sig
        else:
            df['long_support'] = np.exp(s_tl)
            df['long_resist'] = np.exp(r_tl)
            df['long_trend_break_signal'] = sig

    return df
def find_levels( 
        price: np.array, atr: float, # Log closing price, and log atr 
        first_w: float = 0.5, 
        atr_mult: float = 3.0, 
        prom_thresh: float = 0.15
):

    # Setup weights
    last_w = 1.0
    w_step = (last_w - first_w) / len(price)
    weights = first_w + np.arange(len(price)) * w_step
    weights[weights < 0] = 0.0

    # Get kernel of price. 
    kernal = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile
    min_v = np.min(price)
    max_v = np.max(price)
    step = (max_v - min_v) / 200
    price_range = np.arange(min_v, max_v, step)
    pdf = kernal(price_range) # Market profile

    # Find significant peaks in the market profile
    pdf_max = np.max(pdf)
    prom_min = pdf_max * prom_thresh

    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)
    levels = [] 
    for peak in peaks:
        levels.append(np.exp(price_range[peak]))

    return levels, peaks, props, price_range, pdf, weights




def support_resistance_levels(
        data: pd.DataFrame, lookback: int, 
        first_w: float = 0.5, atr_mult:float=3.0, prom_thresh:float =0.15
):

    # Get log average true range, 
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    all_levels = ["None"] * len(data)
    for i in range(lookback, len(data)):
        i_start  = i - lookback
        vals = np.log(data.iloc[i_start+1: i+1]['close'].to_numpy())
        levels, peaks, props, price_range, pdf, weights= find_levels(vals, atr.iloc[i], first_w, atr_mult, prom_thresh)
        all_levels[i] = levels
        
    return all_levels

ticker_data["short_sr_min"] = ["None" if not i else min(i) for i in support_resistance_levels(ticker_data, 25)]
ticker_data["short_sr_max"] = ["None" if not i else max(i) for i in support_resistance_levels(ticker_data, 25)]
ticker_data["long_sr_min"] = ["None" if not i else min(i) for i in support_resistance_levels(ticker_data, 100)]
ticker_data["long_sr_max"] = ["None" if not i else max(i) for i in support_resistance_levels(ticker_data, 100)]
ticker_data['pivot'] = ticker_data.apply(
    lambda x: pivotid(ticker_data, x.name, 4, 4), axis=1)
ticker_data = fit_all_high_low(ticker_data)

ticker_data = Tri_Rec_Broad(ticker_data)
ticker_data = trendline_breakout(ticker_data)


In [47]:
ticker_data.iloc[2250:2290,0:15]

,date,open,high,low,close,adjClose,volume,change,changePercent,vwap,spread,5_day_avg_spread,5_day_avg_volume,EMA_5,EMA_10
2250,2018-12-11,42.9150,42.9475,41.7500,42.1575,40.571247,189126660,-0.7575,-1.77000,42.24,1.1975,1.41100,188817425.6,42.885430,43.715291
2251,2018-12-12,42.6000,42.9800,42.2550,42.2750,40.684334,142510636,-0.3250,-0.76291,42.57,0.7250,1.25000,184244127.2,42.681953,43.453420
2252,2018-12-13,42.6225,43.1425,42.3875,42.7375,41.129425,127591308,0.1150,0.26981,42.74,0.7550,1.18300,175291820.8,42.700469,43.323253
2253,2018-12-14,42.2500,42.2700,41.3200,41.3700,39.813381,162677640,-0.8800,-2.08000,41.66,0.9500,1.06350,174002044.0,42.256979,42.968116
2254,2018-12-17,41.3625,42.0875,40.6825,40.9850,39.442871,176888088,-0.3775,-0.91266,41.36,1.4050,1.00650,159758866.4,41.832986,42.607549
2255,2018-12-18,41.3450,41.8825,41.0975,41.5175,39.955330,135366072,0.1725,0.41722,41.48,0.7850,0.92400,149006748.8,41.727824,42.409359
2256,2018-12-19,41.5000,41.8625,39.7725,40.2225,38.709053,196189188,-1.2775,-3.08000,40.81,2.0900,1.19700,159742459.2,41.226049,42.011748
2257,2018-12-20,40.1000,40.5275,38.8250,39.2075,37.732254,259091840,-0.8925,-2.23000,39.60,1.7025,1.38650,186042565.6,40.553200,41.501885
2258,2018-12-21,39.2150,39.5400,37.4075,37.6825,36.264626,382977536,-1.5325,-3.91000,38.25,2.1325,1.62300,230102544.8,39.596300,40.807451
2259,2018-12-24,37.0375,37.8875,36.6475,36.7075,35.326313,148616928,-0.3300,-0.89099,37.19,1.2400,1.59000,224448312.8,38.633366,40.062005


In [32]:
# ticker_data.dropna(how="all", axis=1, inplace=True)
# ticker_data = ticker_data.loc[:, (ticker_data != 0).any(axis=0)]
ticker_data.dropna()


,date,open,high,low,close,adjClose,volume,change,changePercent,vwap,...,long_resist_slope,Triangle,Rectangle,Broaden,short_support,short_resist,short_trend_break_signal,long_support,long_resist,long_trend_break_signal
199,2010-10-18,11.3739,11.3929,11.2246,11.3571,9.652918,1103638564,-0.0168,-0.14771,11.23,...,0.001614,None,None,None,2.326804,2.424944,0.0,2.112083,2.421848,0.0
200,2010-10-19,10.8357,11.2061,10.7150,11.0532,9.394601,1232280532,0.2175,2.01000,11.02,...,0.001614,None,None,None,2.332227,2.438040,0.0,2.115265,2.434604,0.0
201,2010-10-20,11.0357,11.2232,10.9596,11.0904,9.426168,721624120,0.0547,0.49566,11.11,...,0.001614,None,None,None,2.339935,2.443090,0.0,2.118937,2.436218,0.0
202,2010-10-21,11.1557,11.2407,10.9571,11.0543,9.395511,551457284,-0.1014,-0.90895,11.10,...,0.001614,None,None,None,2.351875,2.448141,0.0,2.123868,2.437832,0.0
203,2010-10-22,11.0382,11.0729,10.9393,10.9811,9.333282,372775424,-0.0571,-0.51729,11.01,...,0.001614,None,None,None,2.364942,2.453190,0.0,2.129984,2.439446,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397,2023-07-05,191.5650,192.9800,190.6200,191.3300,191.330002,46896861,-0.2350,-0.12267,191.58,...,0.002573,None,None,None,5.239322,5.275909,0.0,5.217875,5.275037,0.0
3398,2023-07-06,189.8400,192.0200,189.2000,191.8100,191.809998,45155523,1.9700,1.04000,190.82,...,0.002374,None,None,None,5.242886,5.278697,0.0,5.220787,5.296440,0.0
3399,2023-07-07,191.4100,192.6700,190.2400,190.6800,190.679993,46757498,-0.7300,-0.38138,191.42,...,0.002374,None,None,None,5.246368,5.281489,0.0,5.223710,5.279835,0.0
3400,2023-07-10,189.2600,189.9900,187.0350,188.6100,188.610001,59912163,-0.6500,-0.34344,188.36,...,0.002374,None,None,None,5.249928,5.284280,0.0,5.226645,5.282215,0.0
